# Benchmarking tflite models  

This notebook requires the native benchmark binary for linux that you can get from this page:  
 [https://www.tensorflow.org/lite/performance/measurement](https://www.tensorflow.org/lite/performance/measurement)  

 This binary must be placed into the "benchmarking folder".

 This notebook must be run under LINUX!

 TensorFlow Lite benchmark tools currently measure and calculate statistics for the following important performance metrics:

- Initialization time
- Inference time of warmup state
- Inference time of steady state
- Memory usage during initialization time
- Overall memory usage

In [481]:
import os, sys, math, datetime
import pathlib
from pathlib import Path

# import workbench.config.config
from workbench.config.config import initialize
from workbench.utils.utils import create_filepaths

In [482]:
import re
from matplotlib import pyplot as plt
#import plotly.express as px
import pandas as pd


# enable plotly in VS Studio Code
#import plotly.io as pio
#pio.renderers.default = "notebook_connected"
#pio.renderers.default = "plotly_mimetype+notebook"

import wandb

In [483]:
# Configure pandas to show all columns & rows
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [484]:
models_dir = initialize()

In [485]:
global model_name
#model_name = "efficientNetB0_1_96_c1_o3_keras"
#model_name = "mobilenetv2_0.1_96_c1_o3_l1"
#model_name = "shufflenetv1_1_96_c3_o3_000"
model_name = "shufflenetv1_0.2_96_c3_o3_g2"

models_path, models_summary_path, models_image_path, models_layer_df_path, models_tf_path, models_tflite_path, models_tflite_opt_path = create_filepaths(model_name)

i:\tinyml\tiny_cnn\models


In [486]:
models_benchmark_path = models_dir.joinpath(model_name, f"{model_name}_benchmark.txt")
models_benchmark_path

WindowsPath('i:/tinyml/tiny_cnn/models/shufflenetv1_0.2_96_c3_o3_g2/shufflenetv1_0.2_96_c3_o3_g2_benchmark.txt')

In [487]:
models_tflite_opt_path.as_posix()

'i:/tinyml/tiny_cnn/models/shufflenetv1_0.2_96_c3_o3_g2/shufflenetv1_0.2_96_c3_o3_g2_INT8.tflite'

In [488]:
# ! ./benchmarking/linux_x86-64_benchmark_model \
#     --graph=$models_tflite_opt_path \
#     --num_threads=1 \
#     --enable_op_profiling=true \
#     | tee $models_benchmark_path

# Helper functions

In [489]:

def clean_model_summary(filepath): 
    clean_lines = []
#     # Parse the MLTK model summary to grab important metrics   
    with open(filepath, "r", encoding="latin-1") as f:
        lines = f.readlines() # list containing lines of file
        for line in lines:
            line = line.strip() # remove leading/trailing white spaces
            if len(line)> 0:
                
                clean_lines.append(line)
            else:
                pass
        #columns = [] # To store column names
    return clean_lines

In [490]:
def clean_column_names(df):
    cols = df.columns

    clean_cols = []
    for col in cols:
        col = col.strip()
        col = col.replace("[" , "")
        col = col.replace("]" , "")   
        clean_cols.append(col)
        
    return clean_cols

In [491]:
def string_percent_to_float(x):
    x = str(x).strip("%")
    return float(x)

In [492]:
def remove_tabs(text):
    """removes tabs from a list of strings

    Args:
        text (list(str)): list of strings that contains tabs

    Returns:
        list(str): list of strings without tabs
    """
    split_text= []
    for l in text:
        split_text.append((l.split("\t")))
        
    return split_text

# Parsing the model file

if models_benchmark_path.is_file():
    

In [493]:
if models_benchmark_path.is_file():
    lines = clean_model_summary(models_benchmark_path)
else:
    raise Exception
    print(f"This file does not exist: {models_benchmark_path}")

In [494]:
lines = clean_model_summary(models_benchmark_path)

In [495]:
lines

['STARTING!',
 'Log parameter values verbosely: [0]',
 'Num threads: [1]',
 'Graph: [/mnt/i/tinyml/tiny_cnn/models/shufflenetv1_0.2_96_c3_o3_g2/shufflenetv1_0.2_96_c3_o3_g2_INT8.tflite]',
 'Enable op profiling: [1]',
 '#threads used for CPU inference: [1]',
 'Loaded model /mnt/i/tinyml/tiny_cnn/models/shufflenetv1_0.2_96_c3_o3_g2/shufflenetv1_0.2_96_c3_o3_g2_INT8.tflite',
 'The input model file size (MB): 0.344856',
 'Initialized session in 284.993ms.',
 'Running benchmark for at least 1 iterations and at least 0.5 seconds but terminate if exceeding 150 seconds.',
 'count=103 first=17965 curr=4197 min=4172 max=17965 avg=4865.02 std=1567',
 'Running benchmark for at least 50 iterations and at least 1 seconds but terminate if exceeding 150 seconds.',
 'count=198 first=4535 curr=4861 min=4321 max=7742 avg=4804.89 std=550',
 'Inference timings in us: Init: 284993, First inference: 17965, Warmup (avg): 4865.02, Inference (avg): 4804.89',
 'Note: as the benchmark tool itself affects memory f

In [496]:
for i, line in enumerate(lines):
    #if line.str.contains('= Run Order ='):
    if "Operator-wise Profiling Info for Regular Benchmark Runs:" in line:
        split_line = i
    else:
        pass

In [497]:
#lines[lines_dict["run_order"]]

In [498]:
model_profiling = lines[:split_line]
model_profiling

['STARTING!',
 'Log parameter values verbosely: [0]',
 'Num threads: [1]',
 'Graph: [/mnt/i/tinyml/tiny_cnn/models/shufflenetv1_0.2_96_c3_o3_g2/shufflenetv1_0.2_96_c3_o3_g2_INT8.tflite]',
 'Enable op profiling: [1]',
 '#threads used for CPU inference: [1]',
 'Loaded model /mnt/i/tinyml/tiny_cnn/models/shufflenetv1_0.2_96_c3_o3_g2/shufflenetv1_0.2_96_c3_o3_g2_INT8.tflite',
 'The input model file size (MB): 0.344856',
 'Initialized session in 284.993ms.',
 'Running benchmark for at least 1 iterations and at least 0.5 seconds but terminate if exceeding 150 seconds.',
 'count=103 first=17965 curr=4197 min=4172 max=17965 avg=4865.02 std=1567',
 'Running benchmark for at least 50 iterations and at least 1 seconds but terminate if exceeding 150 seconds.',
 'count=198 first=4535 curr=4861 min=4321 max=7742 avg=4804.89 std=550',
 'Inference timings in us: Init: 284993, First inference: 17965, Warmup (avg): 4865.02, Inference (avg): 4804.89',
 'Note: as the benchmark tool itself affects memory f

In [499]:
model_lines_dict = {}
for i, line in enumerate(model_profiling):
    #if line.str.contains('= Run Order ='):
    if "= Run Order =" in line:
        model_lines_dict["run_order"] = i
    elif "= Top by Computation Time =" in line:
        model_lines_dict["top_by_computation_time"] = i
    elif "The input model file size" in line:
        model_lines_dict["model_file_size"] = i
    elif "Inference timings in us:" in line:
        model_lines_dict["inference_timings"] = i

model_lines_dict

{'model_file_size': 7,
 'inference_timings': 13,
 'run_order': 17,
 'top_by_computation_time': 21}

In [500]:
model_stats = {}

In [501]:
model_size_string = model_profiling[model_lines_dict["model_file_size"]]
model_size_string
model_size= float(model_size_string.split(":")[-1].strip())
model_stats["model_size_MB"] = model_size

In [502]:
inference_string = model_profiling[model_lines_dict["inference_timings"]]
print(inference_string)
inference_strings = inference_string.split(",")
for item in inference_strings:
    item_time = item.split()[-1]
    print(item_time)

model_stats["init_us"] = int(inference_strings[0].split()[-1].strip())
model_stats["first_inference_us"] = int(inference_strings[1].split()[-1].strip())
model_stats["warmup_avg_us"] = float(inference_strings[2].split()[-1].strip())
model_stats["inference_avg_us"] = float(inference_strings[3].split()[-1].strip())


model_stats

Inference timings in us: Init: 284993, First inference: 17965, Warmup (avg): 4865.02, Inference (avg): 4804.89
284993
17965
4865.02
4804.89


{'model_size_MB': 0.344856,
 'init_us': 284993,
 'first_inference_us': 17965,
 'warmup_avg_us': 4865.02,
 'inference_avg_us': 4804.89}

# Operator profiling

In [503]:
operator_profiling = lines[split_line+1:]
operator_profiling

['============================== Run Order ==============================',
 '[node type]\t  [first]\t [avg ms]\t     [%]\t  [cdf%]\t  [mem KB]\t[times called]\t[Name]',
 'CONV_2D\t    0.129\t    0.146\t  3.105%\t  3.105%\t     0.000\t        1\t[shufflenetv1/re_lu_33/Relu;shufflenetv1/batch_normalization_49/FusedBatchNormV3;shufflenetv1/conv2d_33/BiasAdd/ReadVariableOp;shufflenetv1/conv2d_33/BiasAdd;shufflenetv1/conv2d_33/Conv2D1]:0',
 'MAX_POOL_2D\t    0.040\t    0.045\t  0.951%\t  4.056%\t     0.000\t        1\t[shufflenetv1/max_pooling2d_1/MaxPool]:1',
 'AVERAGE_POOL_2D\t    0.018\t    0.021\t  0.437%\t  4.494%\t     0.000\t        1\t[shufflenetv1/average_pooling2d_3/AvgPool]:2',
 'CONV_2D\t    0.552\t    0.609\t 12.968%\t 17.462%\t     0.000\t        1\t[shufflenetv1/re_lu_34/Relu;shufflenetv1/batch_normalization_50/FusedBatchNormV3;shufflenetv1/conv2d_34/BiasAdd/ReadVariableOp;shufflenetv1/conv2d_34/BiasAdd;Conv2D;Conv2D1]:3',
 'SHAPE\t    0.001\t    0.001\t  0.016%\t 17.478%\t 

In [504]:
operator_lines_dict = {}
for i, line in enumerate(operator_profiling):
    #if line.str.contains('= Run Order ='):
    if "= Run Order =" in line:
        operator_lines_dict["run_order"] = i
    elif "= Top by Computation Time =" in line:
        operator_lines_dict["top_by_computation_time"] = i
    elif "= Summary by node type =" in line:
        operator_lines_dict["summary_by_node_type"] = i

operator_lines_dict

{'run_order': 0, 'top_by_computation_time': 222, 'summary_by_node_type': 235}

In [505]:
def get_operator_df(text, name=""):
    df = pd.DataFrame(text)
    df.rename(columns=df.iloc[0, :], inplace=True) 
    df.drop(df.index[0], inplace=True)
    df.columns = clean_column_names(df)
    try:
        df["%"] = df["%"].apply(string_percent_to_float)
    except:
        df["avg %"] = df["avg %"].apply(string_percent_to_float)
    try:
        df["cdf%"] = df["cdf%"].apply(string_percent_to_float)
    except:
        df["cdf %"] = df["cdf %"].apply(string_percent_to_float)
    df["first"] = df["first"].map(float)
    df["avg ms"] = df["avg ms"].map(float)
    df["mem KB"] = df["mem KB"].map(float)
    df["times called"] = df["times called"].map(int)#
    df.Name = name

    return df

In [506]:
def get_node_df(text, name=""):
    df = pd.DataFrame(text)
    df.rename(columns=df.iloc[0, :], inplace=True) 
    df.drop(df.index[0], inplace=True)
    df.columns = clean_column_names(df)
    df["avg %"] = df["avg %"].apply(string_percent_to_float)
    df["cdf %"] = df["cdf %"].apply(string_percent_to_float)
    df["avg ms"] = df["avg ms"].map(float)
    df["mem KB"] = df["mem KB"].map(float)
    df["times called"] = df["times called"].map(int)
    df["count"] = df["count"].map(int)
    df.Name = name

    return df

In [507]:
summary_by_node_type = operator_profiling[operator_lines_dict["summary_by_node_type"] +1:-3]
summary_by_node_type = remove_tabs(summary_by_node_type)
summary_by_node_type

[['[Node type]',
  '  [count]',
  '  [avg ms]',
  '    [avg %]',
  '    [cdf %]',
  '  [mem KB]',
  '[times called]'],
 ['CONV_2D',
  '       33',
  '     3.780',
  '    82.786%',
  '    82.786%',
  '     0.000',
  '       33'],
 ['DEPTHWISE_CONV_2D',
  '       16',
  '     0.376',
  '     8.235%',
  '    91.021%',
  '     0.000',
  '       16'],
 ['ADD',
  '       13',
  '     0.168',
  '     3.679%',
  '    94.700%',
  '     0.000',
  '       13'],
 ['TRANSPOSE',
  '       16',
  '     0.091',
  '     1.993%',
  '    96.693%',
  '     0.000',
  '       16'],
 ['MAX_POOL_2D',
  '        1',
  '     0.044',
  '     0.964%',
  '    97.657%',
  '     0.000',
  '        1'],
 ['RELU',
  '        3',
  '     0.040',
  '     0.876%',
  '    98.533%',
  '     0.000',
  '        3'],
 ['AVERAGE_POOL_2D',
  '        3',
  '     0.035',
  '     0.767%',
  '    99.299%',
  '     0.000',
  '        3'],
 ['RESHAPE',
  '       32',
  '     0.007',
  '     0.153%',
  '    99.452%',
  '     0.000',


In [508]:
summary_by_node_type_df = get_node_df(summary_by_node_type, name=f"Summary by node type - {model_name}")
summary_by_node_type_df

,Node type,count,avg ms,avg %,cdf %,mem KB,times called
1,CONV_2D,33,3.780,82.786,82.786,0.0,33
2,DEPTHWISE_CONV_2D,16,0.376,8.235,91.021,0.0,16
3,ADD,13,0.168,3.679,94.700,0.0,13
4,TRANSPOSE,16,0.091,1.993,96.693,0.0,16
5,MAX_POOL_2D,1,0.044,0.964,97.657,0.0,1
6,RELU,3,0.040,0.876,98.533,0.0,3
7,AVERAGE_POOL_2D,3,0.035,0.767,99.299,0.0,3
8,RESHAPE,32,0.007,0.153,99.452,0.0,32
9,PACK,32,0.007,0.153,99.606,0.0,32
10,STRIDED_SLICE,32,0.004,0.088,99.693,0.0,32


In [509]:
summary_by_node_type_df.Name

'Summary by node type - shufflenetv1_0.2_96_c3_o3_g2'

In [510]:
summary_by_node_type_df.info

<bound method DataFrame.info of             Node type  count  avg ms   avg %    cdf %  mem KB  times called
1             CONV_2D     33   3.780  82.786   82.786     0.0            33
2   DEPTHWISE_CONV_2D     16   0.376   8.235   91.021     0.0            16
3                 ADD     13   0.168   3.679   94.700     0.0            13
4           TRANSPOSE     16   0.091   1.993   96.693     0.0            16
5         MAX_POOL_2D      1   0.044   0.964   97.657     0.0             1
6                RELU      3   0.040   0.876   98.533     0.0             3
7     AVERAGE_POOL_2D      3   0.035   0.767   99.299     0.0             3
8             RESHAPE     32   0.007   0.153   99.452     0.0            32
9                PACK     32   0.007   0.153   99.606     0.0            32
10      STRIDED_SLICE     32   0.004   0.088   99.693     0.0            32
11               MEAN      1   0.004   0.088   99.781     0.0             1
12    FULLY_CONNECTED      1   0.004   0.088   99.869   

In [511]:
top_by_comp_time = operator_profiling[operator_lines_dict["top_by_computation_time"] +1: operator_lines_dict["summary_by_node_type"]-1]
top_by_comp_time = remove_tabs(top_by_comp_time)
#top_by_comp_time

In [512]:
df_operator_top_by_comp_time = get_operator_df(top_by_comp_time, name=f"Top by compute time - {model_name}")
df_operator_top_by_comp_time

,node type,first,avg ms,%,cdf%,mem KB,times called,Name
1,CONV_2D,0.552,0.609,12.968,12.968,0.0,1,Top by compute time - shufflenetv1_0.2_96_c3_o3_g2
2,CONV_2D,0.246,0.263,5.594,18.563,0.0,1,Top by compute time - shufflenetv1_0.2_96_c3_o3_g2
3,CONV_2D,0.173,0.190,4.039,22.602,0.0,1,Top by compute time - shufflenetv1_0.2_96_c3_o3_g2
4,CONV_2D,0.129,0.146,3.105,25.707,0.0,1,Top by compute time - shufflenetv1_0.2_96_c3_o3_g2
5,CONV_2D,0.119,0.134,2.851,28.558,0.0,1,Top by compute time - shufflenetv1_0.2_96_c3_o3_g2
6,CONV_2D,0.130,0.133,2.835,31.393,0.0,1,Top by compute time - shufflenetv1_0.2_96_c3_o3_g2
7,CONV_2D,0.119,0.133,2.825,34.219,0.0,1,Top by compute time - shufflenetv1_0.2_96_c3_o3_g2
8,CONV_2D,0.119,0.132,2.808,37.026,0.0,1,Top by compute time - shufflenetv1_0.2_96_c3_o3_g2
9,CONV_2D,0.119,0.131,2.795,39.822,0.0,1,Top by compute time - shufflenetv1_0.2_96_c3_o3_g2
10,CONV_2D,0.120,0.131,2.787,42.609,0.0,1,Top by compute time - shufflenetv1_0.2_96_c3_o3_g2


In [513]:
run_order = operator_profiling[operator_lines_dict["run_order"] +1: operator_lines_dict["top_by_computation_time"]]
run_order = remove_tabs(run_order)
#run_order

In [514]:
df_operator_run_order = get_operator_df(run_order, name= f"Run order - {model_name}")
#df_operator_run_order.style.set_properties(**{'text-align': 'left'})
df_operator_run_order.head(20)

,node type,first,avg ms,%,cdf%,mem KB,times called,Name
1,CONV_2D,0.129,0.146,3.105,3.105,0.0,1,Run order - shufflenetv1_0.2_96_c3_o3_g2
2,MAX_POOL_2D,0.040,0.045,0.951,4.056,0.0,1,Run order - shufflenetv1_0.2_96_c3_o3_g2
3,AVERAGE_POOL_2D,0.018,0.021,0.437,4.494,0.0,1,Run order - shufflenetv1_0.2_96_c3_o3_g2
4,CONV_2D,0.552,0.609,12.968,17.462,0.0,1,Run order - shufflenetv1_0.2_96_c3_o3_g2
5,SHAPE,0.001,0.001,0.016,17.478,0.0,1,Run order - shufflenetv1_0.2_96_c3_o3_g2
6,STRIDED_SLICE,0.002,0.001,0.031,17.509,0.0,1,Run order - shufflenetv1_0.2_96_c3_o3_g2
7,PACK,0.001,0.002,0.032,17.541,0.0,1,Run order - shufflenetv1_0.2_96_c3_o3_g2
8,RESHAPE,0.001,0.002,0.043,17.584,0.0,1,Run order - shufflenetv1_0.2_96_c3_o3_g2
9,TRANSPOSE,0.026,0.029,0.614,18.198,0.0,1,Run order - shufflenetv1_0.2_96_c3_o3_g2
10,SHAPE,0.000,0.001,0.011,18.208,0.0,1,Run order - shufflenetv1_0.2_96_c3_o3_g2


In [515]:
df_operator_run_order.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220 entries, 1 to 220
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   node type     220 non-null    object 
 1   first         220 non-null    float64
 2   avg ms        220 non-null    float64
 3   %             220 non-null    float64
 4   cdf%          220 non-null    float64
 5   mem KB        220 non-null    float64
 6   times called  220 non-null    int64  
 7   Name          220 non-null    object 
dtypes: float64(5), int64(1), object(2)
memory usage: 13.9+ KB


# Peak memory usage

In [516]:
models_peak_memory_path = models_dir.joinpath(model_name, f"{model_name}_peak_memory")
models_peak_memory_path

WindowsPath('i:/tinyml/tiny_cnn/models/shufflenetv1_0.2_96_c3_o3_g2/shufflenetv1_0.2_96_c3_o3_g2_peak_memory')

In [517]:
# ! ./benchmarking/linux_x86-64_benchmark_model \
#     --graph=$models_tflite_path \
#     --num_threads=1 \
#     --enable_op_profiling=true \
#     | tee $models_benchmark_path

In [518]:
models_tflite_opt_path

WindowsPath('i:/tinyml/tiny_cnn/models/shufflenetv1_0.2_96_c3_o3_g2/shufflenetv1_0.2_96_c3_o3_g2_INT8.tflite')

In [519]:
# ! python i:\\tinyml\\tflite-tools\\tflite_tools.py \
#     -i $models_tflite_opt_path \
#     --csv $models_peak_memory_path 
 

In [520]:
! python i:\tinyml\tflite-tools\tflite_tools.py -i $models_tflite_opt_path --csv $models_peak_memory_path

Writing model analysis to i:\tinyml\tiny_cnn\models\shufflenetv1_0.2_96_c3_o3_g2\shufflenetv1_0.2_96_c3_o3_g2_peak_memory in CSV format


In [521]:
%%capture peak_memory_str

! python i:\tinyml\tflite-tools\tflite_tools.py -i $models_tflite_opt_path

In [522]:
x = peak_memory_str.stdout
peak_memory_lines = x.split("\n")
peak_memory_lines

['Tensor information (weights excluded):',
 '+-----+----------------------------------------------------------------------------------+--------------------+-----------------+',
 '|  Id |                                      Tensor                                      |       Shape        | Size in RAM (B) |',
 '+-----+----------------------------------------------------------------------------------+--------------------+-----------------+',
 '|   0 |                            serving_default_input_2:0                             |   (1, 96, 96, 3)   |          27,648 |',
 '| 114 | shufflenetv1/re_lu_33/Relu;shufflenetv.../BiasAdd;shufflenetv1/conv2d_33/Conv2D1 |  (1, 48, 48, 24)   |          55,296 |',
 '| 115 |                       shufflenetv1/max_pooling2d_1/MaxPool                       |  (1, 24, 24, 24)   |          13,824 |',
 '| 116 |                     shufflenetv1/average_pooling2d_3/AvgPool                     |  (1, 12, 12, 24)   |           3,456 |',
 '| 117 | shufflene

In [523]:
peak_mem_lines_dict = {}
for i, line in enumerate(peak_memory_lines):
    #if line.str.contains('= Run Order ='):
    if "|  Id | " in line:
        peak_mem_lines_dict["header"] = i
    elif "Current peak memory usage:" in line:
        peak_mem_lines_dict["peak_memory"] = i
    elif "Operator execution schedule:" in line:
        peak_mem_lines_dict["operator_schedule"] = i

peak_mem_lines_dict

{'header': 2, 'operator_schedule': 227, 'peak_memory': 452}

In [524]:
operator_schedule = peak_memory_lines[peak_mem_lines_dict["operator_schedule"]: peak_mem_lines_dict["peak_memory"]]
operator_schedule

['Operator execution schedule:',
 '+----------------------------------------------------------------------------------+-------------------------+----------------+',
 '|                              Operator (output name)                              | Tensors in memory (IDs) | Memory use (B) |',
 '+----------------------------------------------------------------------------------+-------------------------+----------------+',
 '| shufflenetv1/re_lu_33/Relu;shufflenetv.../BiasAdd;shufflenetv1/conv2d_33/Conv2D1 |         [0, 114]        |         82,944 |',
 '|                       shufflenetv1/max_pooling2d_1/MaxPool                       |        [114, 115]       |         69,120 |',
 '|                     shufflenetv1/average_pooling2d_3/AvgPool                     |        [115, 116]       |         17,280 |',
 '| shufflenetv1/re_lu_34/Relu;shufflenetv...enetv1/conv2d_34/BiasAdd;Conv2D;Conv2D1 |     [115, 116, 117]     |         40,320 |',
 '|                          shufflenetv1/r

In [525]:
operator_lines =[]
for line in operator_schedule:
    if line.startswith("|"):
        operator_lines.append(line.strip("|"))
#operator_lines

operator_data =[]
for row in operator_lines:
    operator_data_dict = {}
    data =row.split("|")
    #print(data)
    operator_data_dict["operator"] = data[0].strip()
    operator_data_dict["tensor_ids_in_memory"] = data[1].strip()
    operator_data_dict["peak_memory_kb"] = data[2].strip().replace(",", "")
    operator_data.append(operator_data_dict)

operator_data

[{'operator': 'Operator (output name)',
  'tensor_ids_in_memory': 'Tensors in memory (IDs)',
  'peak_memory_kb': 'Memory use (B)'},
 {'operator': 'shufflenetv1/re_lu_33/Relu;shufflenetv.../BiasAdd;shufflenetv1/conv2d_33/Conv2D1',
  'tensor_ids_in_memory': '[0, 114]',
  'peak_memory_kb': '82944'},
 {'operator': 'shufflenetv1/max_pooling2d_1/MaxPool',
  'tensor_ids_in_memory': '[114, 115]',
  'peak_memory_kb': '69120'},
 {'operator': 'shufflenetv1/average_pooling2d_3/AvgPool',
  'tensor_ids_in_memory': '[115, 116]',
  'peak_memory_kb': '17280'},
 {'operator': 'shufflenetv1/re_lu_34/Relu;shufflenetv...enetv1/conv2d_34/BiasAdd;Conv2D;Conv2D1',
  'tensor_ids_in_memory': '[115, 116, 117]',
  'peak_memory_kb': '40320'},
 {'operator': 'shufflenetv1/reshape_32/Shape',
  'tensor_ids_in_memory': '[116, 117, 118]',
  'peak_memory_kb': '26512'},
 {'operator': 'shufflenetv1/reshape_32/strided_slice',
  'tensor_ids_in_memory': '[116, 117, 118, 119]',
  'peak_memory_kb': '26516'},
 {'operator': 'shuff

In [526]:
peak_memory_df = pd.DataFrame.from_dict(operator_data)
peak_memory_df = peak_memory_df.drop([0])
peak_memory_df["peak_memory_kb"] = peak_memory_df["peak_memory_kb"].map(float)/1000.0
peak_memory_df.Name = f"Peak memory - {model_name}"
peak_memory_df

,operator,tensor_ids_in_memory,peak_memory_kb
1,shufflenetv1/re_lu_33/Relu;shufflenetv.../BiasAdd;shufflenetv1/conv2d_33/Conv2D1,"[0, 114]",82.944
2,shufflenetv1/max_pooling2d_1/MaxPool,"[114, 115]",69.120
3,shufflenetv1/average_pooling2d_3/AvgPool,"[115, 116]",17.280
4,shufflenetv1/re_lu_34/Relu;shufflenetv...enetv1/conv2d_34/BiasAdd;Conv2D;Conv2D1,"[115, 116, 117]",40.320
5,shufflenetv1/reshape_32/Shape,"[116, 117, 118]",26.512
...,...,...,...
216,shufflenetv1/re_lu_65/Relu;shufflenetv1/add_25/add,"[328, 329, 316]",4.320
217,shufflenetv1/global_average_pooling2d_1/Mean,"[329, 330]",1.600
218,shufflenetv1/dense_1/MatMul;shufflenetv1/dense_1/BiasAdd,"[330, 331]",0.163
219,StatefulPartitionedCall:01,"[331, 332]",0.006


In [527]:
models_peak_memory_df_path = models_dir.joinpath(model_name, "peak_memory_df")
models_peak_memory_df_path

WindowsPath('i:/tinyml/tiny_cnn/models/shufflenetv1_0.2_96_c3_o3_g2/peak_memory_df')

In [528]:
peak_memory_df.to_pickle(models_peak_memory_df_path)

In [529]:
peak_memory_df["peak_memory_kb"].max()

82.944

In [530]:
tensor_information = peak_memory_lines[:peak_mem_lines_dict["operator_schedule"]]
tensor_information

['Tensor information (weights excluded):',
 '+-----+----------------------------------------------------------------------------------+--------------------+-----------------+',
 '|  Id |                                      Tensor                                      |       Shape        | Size in RAM (B) |',
 '+-----+----------------------------------------------------------------------------------+--------------------+-----------------+',
 '|   0 |                            serving_default_input_2:0                             |   (1, 96, 96, 3)   |          27,648 |',
 '| 114 | shufflenetv1/re_lu_33/Relu;shufflenetv.../BiasAdd;shufflenetv1/conv2d_33/Conv2D1 |  (1, 48, 48, 24)   |          55,296 |',
 '| 115 |                       shufflenetv1/max_pooling2d_1/MaxPool                       |  (1, 24, 24, 24)   |          13,824 |',
 '| 116 |                     shufflenetv1/average_pooling2d_3/AvgPool                     |  (1, 12, 12, 24)   |           3,456 |',
 '| 117 | shufflene

In [531]:
data_lines =[]
for line in tensor_information:
    if line.startswith("|"):
        data_lines.append(line.strip("|"))
#data_lines

split_data =[]
for row in data_lines:
    line_dict = {}
    data =row.split("|")
    #print(data)
    line_dict["tensor_id"] = data[0].strip()
    line_dict["tensor"] = data[1].strip()
    line_dict["shape"] = data[2].strip()
    line_dict["size_in_RAM_kb"] = data[3].strip().replace(",", "")
    split_data.append(line_dict)

split_data


[{'tensor_id': 'Id',
  'tensor': 'Tensor',
  'shape': 'Shape',
  'size_in_RAM_kb': 'Size in RAM (B)'},
 {'tensor_id': '0',
  'tensor': 'serving_default_input_2:0',
  'shape': '(1, 96, 96, 3)',
  'size_in_RAM_kb': '27648'},
 {'tensor_id': '114',
  'tensor': 'shufflenetv1/re_lu_33/Relu;shufflenetv.../BiasAdd;shufflenetv1/conv2d_33/Conv2D1',
  'shape': '(1, 48, 48, 24)',
  'size_in_RAM_kb': '55296'},
 {'tensor_id': '115',
  'tensor': 'shufflenetv1/max_pooling2d_1/MaxPool',
  'shape': '(1, 24, 24, 24)',
  'size_in_RAM_kb': '13824'},
 {'tensor_id': '116',
  'tensor': 'shufflenetv1/average_pooling2d_3/AvgPool',
  'shape': '(1, 12, 12, 24)',
  'size_in_RAM_kb': '3456'},
 {'tensor_id': '117',
  'tensor': 'shufflenetv1/re_lu_34/Relu;shufflenetv...enetv1/conv2d_34/BiasAdd;Conv2D;Conv2D1',
  'shape': '(1, 24, 24, 40)',
  'size_in_RAM_kb': '23040'},
 {'tensor_id': '118',
  'tensor': 'shufflenetv1/reshape_32/Shape',
  'shape': '(4,)',
  'size_in_RAM_kb': '16'},
 {'tensor_id': '119',
  'tensor': 'sh

In [532]:
tensor_df = pd.DataFrame.from_dict(split_data)
tensor_df = tensor_df.drop([0])
tensor_df["size_in_RAM_kb"] = tensor_df["size_in_RAM_kb"].map(float)/1000.0
tensor_df.Name = f"Tensor information - {model_name}"
tensor_df

,tensor_id,tensor,shape,size_in_RAM_kb
1,0,serving_default_input_2:0,"(1, 96, 96, 3)",27.648
2,114,shufflenetv1/re_lu_33/Relu;shufflenetv.../BiasAdd;shufflenetv1/conv2d_33/Conv2D1,"(1, 48, 48, 24)",55.296
3,115,shufflenetv1/max_pooling2d_1/MaxPool,"(1, 24, 24, 24)",13.824
4,116,shufflenetv1/average_pooling2d_3/AvgPool,"(1, 12, 12, 24)",3.456
5,117,shufflenetv1/re_lu_34/Relu;shufflenetv...enetv1/conv2d_34/BiasAdd;Conv2D;Conv2D1,"(1, 24, 24, 40)",23.040
...,...,...,...,...
217,329,shufflenetv1/re_lu_65/Relu;shufflenetv1/add_25/add,"(1, 3, 3, 160)",1.440
218,330,shufflenetv1/global_average_pooling2d_1/Mean,"(1, 160)",0.160
219,331,shufflenetv1/dense_1/MatMul;shufflenetv1/dense_1/BiasAdd,"(1, 3)",0.003
220,332,StatefulPartitionedCall:01,"(1, 3)",0.003


In [533]:
tensor_df.Name

'Tensor information - shufflenetv1_0.2_96_c3_o3_g2'

In [534]:
tensor_info_df_path = models_dir.joinpath(model_name, "tensor_info_df")
tensor_info_df_path

WindowsPath('i:/tinyml/tiny_cnn/models/shufflenetv1_0.2_96_c3_o3_g2/tensor_info_df')

In [535]:
tensor_df.to_pickle(tensor_info_df_path)

## Parse peak memory

In [536]:
peak_memory_line = peak_memory_lines[peak_mem_lines_dict["peak_memory"]]
peak_memory_line

'Current peak memory usage: 82,944 B'

In [537]:
peak_memory_kb = float(peak_memory_line.split(":")[-1].strip().split()[0].replace(",", ".") )
peak_memory_kb

82.944

In [538]:
model_stats["peak_memory_kb"] = peak_memory_kb

In [539]:
model_stats

{'model_size_MB': 0.344856,
 'init_us': 284993,
 'first_inference_us': 17965,
 'warmup_avg_us': 4865.02,
 'inference_avg_us': 4804.89,
 'peak_memory_kb': 82.944}

# Logging to wandb

In [540]:
# Generate run ids
id = wandb.util.generate_id()

PROJECT = model_name.split("_")[0]

run = wandb.init(
        # Set the project where this run will be logged
        project=PROJECT, 
        name = model_name,
        id = id, 
        resume="allow",
        sync_tensorboard=True
        )
# Specify the configuration variables
config = wandb.config
config.architecture = model_name

# # Create a table
operator_top_by_time_table = wandb.Table(dataframe=df_operator_top_by_comp_time)
# operator_top_by_time_artifact = wandb.Artifact(name=f'{model_name}_operator_top_by_time', type='dataframe')
# operator_top_by_time_artifact.add(operator_top_by_time_table, "Profiling operators - top by time")
# run.log_artifact(operator_top_by_time_artifact)

summary_by_node_type_table = wandb.Table(dataframe=summary_by_node_type_df)
# summary_by_node_type_artifact = wandb.Artifact(name=f'{model_name}_summary_by_node_type', type='dataframe')
# summary_by_node_type_artifact.add(summary_by_node_type_table, "Node type summary")
# run.log_artifact(summary_by_node_type_artifact)

operator_run_order_table = wandb.Table(dataframe=df_operator_run_order)
# operator_run_order_artifact = wandb.Artifact(name=f'{model_name}_operator_run_order', type='dataframe')
# operator_run_order_artifact.add(operator_run_order_table, "Run order")
# run.log_artifact(operator_run_order_artifact)


run.log({"Profiling operators - top by time": operator_top_by_time_table})
run.log({"Node type summary" : summary_by_node_type_table})
run.log({"Run order" : operator_run_order_table})
# run.log({"Chart_table": table})

# table = wandb.Table(columns = ["plotly_figure"])

# # Create path for Plotly figure
# path_to_plotly_html = "./plotly_figure.html"

# # Write Plotly figure to HTML
# fig.write_html(path_to_plotly_html, auto_play = False) # Setting auto_play to False prevents animated Plotly charts from playing in the table automatically

# # Add Plotly figure as HTML file into Table
# table.add_data(wandb.Html(path_to_plotly_html))

# # Log Table
# run.log({"Chart_table": table})

wandb.log(model_stats)

# #wandb.log({"Peak activations chart": fig})

wandb.finish()